# Noise Removal
This notebook deals with applied methods of noise removal. To better understand the methods presented here, read the "Random Noise", "Periodic Noise", and the notebooks in the "Filtering" folder first.

## Section 0. Preparing the Notebook

We start by importing the necessary libraries and then loading sample images to work on.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# importing necessary packages
import numpy as np
from numpy import random
import cv2 as cv
from matplotlib import pyplot as plt
from typing import Union, Iterable
from numpy.lib.stride_tricks import sliding_window_view
from solutions import *
from utils import *

In [ ]:
# loading the sample image and setting the colormap for pyplot
image = np.float64(cv.imread('data/houses.pgm', cv.IMREAD_GRAYSCALE) / 255)
plt.set_cmap('Greys_r')
_ = plt.imshow(image), plt.axis('off')

## Section 1. A New approach to image filtering
Up until now, most of the filtering we did was by either multiplying Fourier transforms, or by using OpenCV's filter function. However, we can use one of NumPy's functions to get a sliding window's view of the image, and apply more varied methods of filtering to the image.

### Section 1.1. Padding the Image

Before we start filtering an image, we should first pad the image properly, so that the output image will have the same size as the input. So let's start by implementing a padding function. You can use any type of padding that you want, but the reference function uses the nearest pixel in the image to fill in the padding pixels.

In [ ]:
def pad(image : np.ndarray, padding : Union[int, Iterable[int]]) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the padding will be applied. It should be a np.ndarray
        with dtype=float64, and with values within [0 1].
    - padding : Union[int, Iterable[int]]
        The amount of padding that should be applied to the image. Either an
        Iterable of 4 integers, in which case each integers determines the
        number of padding pixels that are to be added to each side, or an integer,
        in which case the same amount will be applied to all four side of the
        image. All the values should be non-negative integers.
        The pattern for Iterable paddings is [up, down, left, right].
    Returns:
    - output : np.ndarray
        The padded image, with dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Padding the sample image
padding = 2
image_padded = pad(image, padding)
image_padded_ref = padRef(image, padding)

# Showing the results
_ = plt.figure(figsize=(14, 7))
_ = plt.subplot(1, 2, 1), plt.imshow(image_padded, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Padded Image')
_ = plt.subplot(1, 2, 2), plt.imshow(image_padded_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Padded Image')

### Section 1.2. Getting the Sliding Kernel View
Now, using ``numpy.lib.stride_tricks.sliding_window_view``, get a *kerneled* view of a padded image. Write the parameters for the padding so that the output will have correct dimensions.

In [ ]:
def kernelView(image : np.ndarray, kernel_shape : Iterable[int]) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image whose sliding window view will be viewed. Should be an np.ndarray
        with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    Returns:
    - view : np.ndarray
        The sliding window view, which should be an np.ndarray of 4 dimensions
        with dtype=np.float64.
    """
    # ====== YOUR CODE ======

kernel_shape = (3, 3)
image_kernel_view = kernelView(image, kernel_shape)
image_kernel_view_ref = kernelViewRef(image, kernel_shape)

# Checking the dimensions
print(f'Dimensions of the original image: {image.shape}')
print(f'Dimensions of your view: {image_kernel_view.shape}')
print(f'Dimensions of the reference view: {image_kernel_view_ref.shape}')

### Section 1.3. Applying a Kernel to The View
Now that we have a view of the sliding image, we can use it to directly apply a kernel to our image. While doing this is best avoided, since using OpenCV or dedicated functions is more efficient, there is no harm in trying a new way of applying filters, and getting familiar with how things are done in NumPy. You can try applying a mean filter using only NumPy functions.

In [ ]:
def applyKernel(view : np.ndarray, kernel : np.ndarray) -> np.ndarray:
    """
    Parameters:
    - view : np.ndarray
        The sliding window view, which should be an np.ndarray of 4 dimensions
        with dtype=np.float64.
    - kernel : np.ndarray
        A 2-D kernel, with dimensions equal to the last dimensions of view.
    Returns:
    - image : np.ndarray
        The output image of applying the kernel.
    """

# Creating a random kernel
kernel_shape = (3, 3)
kernel = np.float64(random.rand(*kernel_shape)) - 0.5

# Applying the kernel
view = kernelViewRef(image, kernel_shape)
image_filtered_kernel = applyKernel(view, kernel)
image_filtered_kernel_ref = applyKernelRef(view, kernel)

# Showing the results
_ = plt.figure(figsize=(14, 7))
_ = plt.subplot(1, 2, 1), plt.imshow(image_filtered_kernel), plt.axis('off'), plt.title('Your Filtered Image')
_ = plt.subplot(1, 2, 2), plt.imshow(image_filtered_kernel_ref), plt.axis('off'), plt.title('Reference Filtered Image')

### Section 1.4. Geometric Mean Filter
The mean filter that we implemented before is also known as the *arithmetic mean* filter. There is, however, another type of mean value for a set of values, which is known as the geometric mean; and is equal to the $n$th root of the product of $n$ values.

$
\LARGE [\prod_{i=1}^n]^{\frac{1}{n}}
$

While the geometric mean is known to diminish the effect of outliers compared to the arithmetic mean, it is sensitive to zero values. Try implementing this filter using the windowed view.

In [ ]:
def geometricMeanFilter(image : np.ndarray, kernel_shape : Iterable[int]) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the geometric mean filter will be applied. Should be an 
        np.ndarray with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    Returns:
    - output : np.ndarray
        The filtered image, which should be an 2-dimensional np.ndarray with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Creating a noisy image
sigma = 0.05
image_gaussian_noise = addGaussianNoiseRef(image, sigma)

# Filtering the image
kernel_size = (3, 3)
image_gaussian_geometric_filtered = geometricMeanFilter(image_gaussian_noise, kernel_size)
image_gaussian_geometric_filtered_ref = geometricMeanFilterRef(image_gaussian_noise, kernel_size)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(image_gaussian_geometric_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Filtered Image')
_ = plt.subplot(2, 2, 2), plt.imshow(image_gaussian_geometric_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Filtered Image')
_ = plt.subplot(2, 2, 3), plt.imshow(image_gaussian_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Noisy Image')
_ = plt.subplot(2, 2, 4), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

## Section 2. Order-Statistic Filters
Having the windowed view of the image, we are now free to apply all kinds of mathematical operations on these windows. We will go through some order-statistic methods in this section, which use the statistical properties of the window to filter the image.

### Section 2.1. Median Filtering
One of the basic, yet very effective methods of order-statistic filtering, is outputing the median of each window. This method is especially useful for removing the types of noise which cause extreme changes in pixel intesity, e.g. salt & pepper noise, or pixels whose value has been greatly altered due to noise in the communication channel. Implement a median filter function, and see its effects on salt & pepper noise.

In [ ]:
def medianFilter(image : np.ndarray, kernel_shape : Iterable[int]) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the median filter will be applied. Should be an np.ndarray
        with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    Returns:
    - output : np.ndarray
        The filtered image, which should be an 2-dimensional np.ndarray with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Creating a noisy image
p = 0.1
image_snp_noise = addSNPNoiseRef(image, p)

# Filtering the image
kernel_size = (3, 3)
image_snp_median_filtered = medianFilter(image_snp_noise, kernel_size)
image_snp_median_filtered_ref = medianFilterRef(image_snp_noise, kernel_size)
image_snp_mean_filtered_ref = cv.blur(image_snp_noise, kernel_size)
image_snp_gaussian_filtered_ref = cv.GaussianBlur(image_snp_noise, kernel_size, 1)

# Showing the results
_ = plt.figure(figsize=(18, 12))
_ = plt.subplot(2, 3, 1), plt.imshow(image_snp_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Noisy Image')
_ = plt.subplot(2, 3, 2), plt.imshow(image_snp_median_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Filtered Image')
_ = plt.subplot(2, 3, 3), plt.imshow(image_snp_median_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Filtered Image')
_ = plt.subplot(2, 3, 4), plt.imshow(image_snp_mean_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Mean Filtered Image')
_ = plt.subplot(2, 3, 5), plt.imshow(image_snp_gaussian_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Gaussian Filtered Image')
_ = plt.subplot(2, 3, 6), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

### Section 2.2. Alpha-Trimmed Mean Filter
As you saw, the median filter can be pretty handy in removing salt and pepper noise. However, it tends to ignore all the non-noise pixels in a window, which causes rounding in corners and removal of some image details. We can use our windowed view to remove a fraction of brightest and darkest pixels, and output the average value of the rest of the pixels. The fraction of removed pixels is denoted by $\alpha$, hence the name of the filter. Implement this filter and see how its output on a doubly-noised image compare with naive median and mean filters.

In [ ]:
def alphaTrimmedMeanFilter(image : np.ndarray, kernel_shape : Iterable[int], alpha : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the alpha-trimmed mean filter will be applied. Should be an 
        np.ndarray with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    - alpha : float [0 1)
        The alpha parameter in the alpha-trimmed mean. Should be a non-negative number
        less than 1.
    Returns:
    - output : np.ndarray
        The filtered image, which should be an 2-dimensional np.ndarray with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Creating a image with S&P and Gaussian noise
sigma = 0.1
p = 0.1
image_double_noise = addGaussianNoiseRef(image, sigma)
image_double_noise = addSaltNoiseRef(image_double_noise, p)

# Filtering the image
kernel_size = (3, 3)
alpha = 0.3
image_alpha_filtered = alphaTrimmedMeanFilter(image_double_noise, kernel_size, alpha)
image_alpha_filtered_ref = alphaTrimmedMeanFilterRef(image_double_noise, kernel_size, alpha)
image_mean_filtered_ref = cv.blur(image_double_noise, kernel_size)
image_median_filtered_ref = medianFilterRef(image_double_noise, kernel_size)

# Showing the results
_ = plt.figure(figsize=(18, 12))
_ = plt.subplot(2, 3, 1), plt.imshow(image_double_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Noisy Image')
_ = plt.subplot(2, 3, 2), plt.imshow(image_alpha_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Filtered Image')
_ = plt.subplot(2, 3, 3), plt.imshow(image_alpha_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Filtered Image')
_ = plt.subplot(2, 3, 4), plt.imshow(image_mean_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Mean Filtered Image')
_ = plt.subplot(2, 3, 5), plt.imshow(image_mean_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Median Filtered Image')
_ = plt.subplot(2, 3, 6), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

### Section 2.3. Midpoint Filter
Another way of mixing order-statistic and mean filtering, is by finding the midpoint between a kernel's maximum and minimum values. This type of filtering is mostly effective when it comes to randomly distributed noise like Gaussian or uniform noise. Applying this filter to an image with salt and pepper noise would further deteriorate its quality.

In [ ]:
def midpointFilter(image : np.ndarray, kernel_shape : Iterable[int]) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the midpoint filter will be applied. Should be an np.ndarray
        with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    Returns:
    - output : np.ndarray
        The filtered image, which should be an 2-dimensional np.ndarray with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Creating a image with Gaussian noise
sigma = 0.1
image_gaussian_noise = addGaussianNoiseRef(image, sigma)

# Filtering the image
kernel_size = (3, 3)
alpha = 0.3
image_midpoint_filtered = midpointFilter(image_gaussian_noise, kernel_size)
image_midpoint_filtered_ref = midpointFilterRef(image_gaussian_noise, kernel_size)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(image_midpoint_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Filtered Image')
_ = plt.subplot(2, 2, 2), plt.imshow(image_midpoint_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Filtered Image')
_ = plt.subplot(2, 2, 3), plt.imshow(image_gaussian_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Noisy Image')
_ = plt.subplot(2, 2, 4), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

## Section 3. Harmonic and Contraharmonic Filters

### Section 3.1. Harmonic Filter
A harmonic filter sums the inverses of pixels and then inverts the average result. Such an operation, as described below, can be very effective against salt noise, but will amplify dark pixels. In the notation below, $m$ and $n$ are the dimensions of the kernel $K$.

$
\hat{f}(i,j) = mn [\large \sum_{(r,c) \in K_{ij}} \normalsize f(r,c)^{-1}] ^{-1}
$

Implement this filter and see its results on two images, one with salt, and other with salt and pepper noise. Note that your implementation should not raise a zero division error.

In [ ]:
def harmonicFilter(image : np.ndarray, kernel_shape : Iterable[int]) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the harmonic filter will be applied. Should be an np.ndarray
        with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    Returns:
    - output : np.ndarray
        The filtered image, which should be an 2-dimensional np.ndarray with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Creating a pair of noisy images
p = 0.1
image_salt_noise = addSaltNoiseRef(image, p)
image_snp_noise = addSNPNoiseRef(image, p)

# Filtering the images
kernel_size = (3, 3)
image_salt_harmonic_filtered = harmonicFilter(image_salt_noise, kernel_size)
image_salt_harmonic_filtered_ref = harmonicFilterRef(image_salt_noise, kernel_size)
image_snp_harmonic_filtered_ref = harmonicFilterRef(image_snp_noise, kernel_size)

# Showing the results
_ = plt.figure(figsize=(18, 12))
_ = plt.subplot(2, 3, 1), plt.imshow(image_salt_harmonic_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Salt Noise Filtered Image')
_ = plt.subplot(2, 3, 2), plt.imshow(image_salt_harmonic_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Salt Noise Filtered Image')
_ = plt.subplot(2, 3, 3), plt.imshow(image_snp_harmonic_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference S&P Noise Filtered Image')
_ = plt.subplot(2, 3, 4), plt.imshow(image_salt_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Image with Salt Noise')
_ = plt.subplot(2, 3, 5), plt.imshow(image_snp_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Image with S&P Noise')
_ = plt.subplot(2, 3, 6), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

### Section 3.2. Contraharmonic Filter
A generalization of the harmonic filter is the contraharmonic filter, where the sum of $Q+1$th power of a kernel's pixels is divided by the sum of their $Q$th power. By taking a look at the equation for the harmonic filter, you can see that for $Q=-1$, it is identical to the harmonic filter; and for $Q=-1$, it becomes identical to the arithmetic mean filter.

$
\hat{f}(i,j) = \frac{\LARGE \sum_{(r,c) \in K_{ij}} \Large f(r,c)^{Q+1}}{\LARGE \sum_{(r,c) \in K_{ij}} \Large f(r,c)^{Q}}
$

implement this filter and see its results for different $k$ values. As a general rule of thumb, $Q \leq -1$ will cause behavior similar to the harmonic filter when it comes to dark and bright pixels. For positive values of $Q$, the situation will be reversed.

In [ ]:
def contraharmonicFilter(image : np.ndarray, kernel_shape : Iterable[int], Q : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the contraharmonic filter will be applied. Should be an
        np.ndarray with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    - Q : float
        The Q parameter in the contraharmonic filter.
    Returns:
    - output : np.ndarray
        The filtered image, which should be an 2-dimensional np.ndarray with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Creating a painr of noisy images
p = 0.1
image_salt_noise = addSaltNoiseRef(image, p)
image_pepper_noise = addPepperNoiseRef(image, p)

# Filtering the images
kernel_size = (3, 3)
Q = 2.5
image_salt_contraharmonic_filtered = contraharmonicFilter(image_salt_noise, kernel_size, Q)
image_salt_contraharmonic_filtered_ref = contraharmonicFilterRef(image_salt_noise, kernel_size, Q)
image_pepper_contraharmonic_filtered = contraharmonicFilter(image_pepper_noise, kernel_size, Q)
image_pepper_contraharmonic_filtered_ref = contraharmonicFilterRef(image_pepper_noise, kernel_size, Q)

# Showing the results
_ = plt.figure(figsize=(18, 12))
_ = plt.subplot(2, 3, 1), plt.imshow(image_salt_contraharmonic_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Salt Noise Filtered Image')
_ = plt.subplot(2, 3, 2), plt.imshow(image_salt_contraharmonic_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Salt Noise Filtered Image')
_ = plt.subplot(2, 3, 3), plt.imshow(image_salt_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Salt Noise Image')
_ = plt.subplot(2, 3, 4), plt.imshow(image_pepper_contraharmonic_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Pepper Noise Filtered Image')
_ = plt.subplot(2, 3, 5), plt.imshow(image_pepper_contraharmonic_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Pepper Noise Filtered Image')
_ = plt.subplot(2, 3, 6), plt.imshow(image_pepper_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Pepper Noise Image')

## Section 4. Edge Preserving Filters

### Section 4.1. Adaptive Filtering
One way to counter the edge destruction that results from low-pass filters, is decreasing the intensity of the filtering in areas where edges are present. One way of finding edge areas is by measuring the variance in a window, and comparing it to a baseline value for noise variance. Since the variance of areas with details is higher, we will have a less intense filter there. Here, the equation for one window is shown. Note that $\sigma_{\eta}^2$ is the estimated variance of the added noise, and $\sigma_{K}^2$ is the variance of the current kernel $K$.

$
\hat{K} = K - \frac{\sigma_{\eta}^2}{\sigma_{K}^2} (K - \bar{K})
$

Note that any low-pass filter can be used instead of the mean filter which is used above ($\bar{K}$). Try implementing this filter and see its results on the edges of a sample image.

**Note:** While statistically, we expect the $\sigma$ of the sum of two variables to be higher than either of their $\sigma$'s, it's best practice to set a higher cap of $1$ on the $\frac{\sigma_{\eta}^2}{\sigma_{K}^2}$ part of the equation for the rare instance where the $\sigma$ of a kernel is lower than that of the noise.

In [ ]:
def adaptiveFilter(image : np.ndarray, kernel_shape : Iterable[int], base_std : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the adaptive filter will be applied. Should be an np.ndarray
        with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    - base_variance : float [0 inf)
        An estimation of the standard deviation for additive noise. Should be a
        non-negative number.
    Returns:
    - output : np.ndarray
        The filtered image, which should be an 2-dimensional np.ndarray with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Creating a noisy image
sigma = 0.1
image_gaussian_noise = addGaussianNoiseRef(image, sigma)

# Filtering the images
base_std = 0.1
kernel_size = (3, 3)
image_gaussian_adaptive_filtered = adaptiveFilter(image_gaussian_noise, kernel_size, base_std)
image_gaussian_adaptive_filtered_ref = adaptiveFilterRef(image_gaussian_noise, kernel_size, base_std)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(image_gaussian_adaptive_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Adaptive Filtered Image')
_ = plt.subplot(2, 2, 2), plt.imshow(image_gaussian_adaptive_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Adaptive Filtered Image')
_ = plt.subplot(2, 2, 3), plt.imshow(image_gaussian_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Noisy Image')
_ = plt.subplot(2, 2, 4), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

### Section 4.2. Bilateral Filter
Another way of preventing edge destruction in images, is by taking into account both the spatial and the intensity difference of neighboring pixels when calculating the output. This method is generally known as the bilateral filter, and is defined as the following equation. Note that $\boldsymbol{c}$ is the central position, and $\boldsymbol{p}$ is a position vector within the kernel.

$
\hat{K} = \Large \frac{1}{W} \sum_{\boldsymbol{p} \in K} \normalsize f_r(\parallel \boldsymbol{p} - \boldsymbol{c} \parallel) g_s(\parallel K(\boldsymbol{p}) - K(\boldsymbol{c}) \parallel) K(\boldsymbol{p})
$

$
W = \Large \sum_{\boldsymbol{p} \in K} \normalsize f_r(\parallel \boldsymbol{p} - \boldsymbol{c} \parallel) g_s(\parallel K(\boldsymbol{p}) - K(\boldsymbol{c}) \parallel)
$

Both $g_r$ and $f_r$ are arbitrary functions, generally with positive decreasing values for non-negative inputs. One such function would be a Gaussian function, but any other function with such a property can be used in its stead, as long as it gives an acceptable output. The $W$ is simply the sum of coefficients, and is used for normalization.
Try implementing a bilateral filter with gaussian smoothing functions and see its results on a sample noisy image.

In [ ]:
def bilateralFilter(image : np.ndarray, kernel_shape : Iterable[int], spatial_sigma : float, intensity_sigma : float) -> np.ndarray:
    """
    Parameters:
    - image : np.ndarray
        An image on which the bilateral filter will be applied. Should be an np.ndarray
        with dtype=np.float64.
    - kernel_shape : Iterable[int]
        An Iterable of two positive integers which determines the size of the kernel.
    - spatial_sigma : float [0 inf)
        The sigma value for the spatial smoothing fucntion. Should be a non-negative
        number.
    - intensity_sigma : float [0 inf)
        The sigma value for the intensity smoothing fucntion. Should be a non-negative
        number.
    Returns:
    - output : np.ndarray
        The filtered image, which should be an 2-dimensional np.ndarray with
        dtype=np.float64.
    """
    # ====== YOUR CODE ======
    raise NotImplementedError()

# Creating a noisy image
sigma = 0.1
image_gaussian_noise = addGaussianNoiseRef(image, sigma)

# Filtering the images
spatial_sigma = 3
intensity_sigma = 0.1
kernel_size = (7, 7)
image_gaussian_bilateral_filtered = bilateralFilter(image_gaussian_noise, kernel_size, spatial_sigma, intensity_sigma)
image_gaussian_bilateral_filtered_ref = bilateralFilterRef(image_gaussian_noise, kernel_size, spatial_sigma, intensity_sigma)

# Showing the results
_ = plt.figure(figsize=(15, 15))
_ = plt.subplot(2, 2, 1), plt.imshow(image_gaussian_bilateral_filtered, vmin=0, vmax=1), plt.axis('off'), plt.title('Your Bilateral Filtered Image')
_ = plt.subplot(2, 2, 2), plt.imshow(image_gaussian_bilateral_filtered_ref, vmin=0, vmax=1), plt.axis('off'), plt.title('Reference Bilateral Filtered Image')
_ = plt.subplot(2, 2, 3), plt.imshow(image_gaussian_noise, vmin=0, vmax=1), plt.axis('off'), plt.title('Noisy Image')
_ = plt.subplot(2, 2, 4), plt.imshow(image, vmin=0, vmax=1), plt.axis('off'), plt.title('Original Image')

# Scratchpad
You can use this section as a scratchpad, without making a mess in the notebook. :)